In [13]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../../MNIST_data',one_hot=True)
#mnist = input_data.read_data_sets(FLAGS.data_dir,one_hot=True)

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
from tensorflow.contrib import rnn

In [3]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

timesteps = 28
n_input = 28
n_class = 10
n_hidden = 128
batch_size = 64

X = tf.placeholder(tf.float32, shape=[None, timesteps, n_input])
Y = tf.placeholder(tf.float32, shape=[None, n_class])

with tf.name_scope('weights'):
    weights = {
        'W': tf.Variable(tf.random_normal([n_hidden, n_class]))
    }
    
    variable_summaries(weights['W'])
    
with tf.name_scope('biases'):
    biases ={
        'b': tf.Variable(tf.random_normal([n_class]))
    }
    variable_summaries(biases['b'])

In [4]:
def RNN(x, weights, biases):
    x = tf.unstack(x, timesteps, 1)
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    outputs,states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['W'])+biases['b']

In [5]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

with tf.name_scope('cross_entropy'):
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
    tf.summary.scalar('cross_entropy', loss_op)

train = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss_op)

correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(Y,1))
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar('accuracy', accuracy)
#correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(Y,1))
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [16]:
sess = tf.Session()
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('/Users/dmitriykhvan/git_repo/kaggle/Tensorflow-Tutorials' + '/train',sess.graph)
sess.run(tf.global_variables_initializer())

for step in range(1, 10001):
    batch_X, batch_Y = mnist.train.next_batch(batch_size)
    #print(batch_Y.shape)
    batch_X = batch_X.reshape((batch_size, timesteps, n_input))
    summary, _=sess.run([train,merged], feed_dict={X: batch_X, Y: batch_Y})
    #summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
    train_writer.add_summary(summary, step)
    if step % 200 == 0 or step == 1:
        loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_X, Y: batch_Y})
        print("Step " + str(step) + ", Minibatch loss: " + "{:.4f}".format(loss) + \
             ", Training accuracy: "+ "{:.3f}".format(acc))

Step 1, Minibatch loss: 2.8800, Training accuracy: 0.109
Step 200, Minibatch loss: 2.1655, Training accuracy: 0.219
Step 400, Minibatch loss: 1.9598, Training accuracy: 0.344
Step 600, Minibatch loss: 1.8071, Training accuracy: 0.484
Step 800, Minibatch loss: 1.7292, Training accuracy: 0.453
Step 1000, Minibatch loss: 1.5486, Training accuracy: 0.500
Step 1200, Minibatch loss: 1.3238, Training accuracy: 0.656
Step 1400, Minibatch loss: 1.3316, Training accuracy: 0.656
Step 1600, Minibatch loss: 1.3546, Training accuracy: 0.594
Step 1800, Minibatch loss: 1.4990, Training accuracy: 0.500
Step 2000, Minibatch loss: 1.4177, Training accuracy: 0.594
Step 2200, Minibatch loss: 1.1979, Training accuracy: 0.656
Step 2400, Minibatch loss: 1.1630, Training accuracy: 0.641
Step 2600, Minibatch loss: 1.0892, Training accuracy: 0.688
Step 2800, Minibatch loss: 1.0096, Training accuracy: 0.656
Step 3000, Minibatch loss: 1.1598, Training accuracy: 0.594
Step 3200, Minibatch loss: 1.0490, Training acc

KeyboardInterrupt: 